In [4]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [5]:
gmsh.open("furat.geo")

Info    : Reading 'furat.geo'...
Info    : Meshing 1D...                                                                                                                        
Info    : [  0%] Meshing curve 5 (Ellipse)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 70%] Meshing curve 8 (Line)
Info    : [ 90%] Meshing curve 9 (Line)
Info    : Done meshing 1D (Wall 0.00118934s, CPU 0.001182s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Blossom: 22788 internal 756 closed
Info    : Blossom recombination completed (Wall 0.875629s, CPU 0.866532s): 7553 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.805118, min Q = 0.312842
Info    : Done meshing 2D (Wall 1.41433s, CPU 1.39668s)
Info    : Meshing order 4 (curvilinear on)...
Info    : [  0%] Meshing curve 5 order 4
Info    : [ 20%] Meshing curve 6 order 4
Info    : [ 40%] Meshing curve 7 order 4
Info    : [ 60%] Meshing curve 8 order

In [6]:
mat = FEM.material("plate", E=2e5, ν=0.3)
problem = FEM.Problem([mat], type=:PlaneStress)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 5365)


LowLevelFEM.Problem("furat", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("plate", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 122360)

In [7]:
supp1 = FEM.displacementConstraint("left", ux=0, uy=0)
supp2 = FEM.displacementConstraint("right", uy=0)
load = FEM.load("right", fx=1)
loadL = FEM.load("left", fy=-1)
loadR = FEM.load("right", fy=1)

("right", 0, 1, 0)

In [8]:
q = FEM.solveDisplacement(problem, [load], [supp1, supp2])

244720-element Vector{Float64}:
  0.002575733259242966
  4.9038817107401e-10
  0.0
  0.0
  0.004956911004767871
  0.0
  0.0
  0.0
  0.004956910781830732
  0.0
  ⋮
 -2.1337219889981666e-5
  0.002463063685535801
 -2.1214675034023918e-5
  0.002463167001644638
 -2.138847892739162e-5
  0.002463908730593332
 -2.1506566689757308e-5
  0.0024634843526827525
 -2.1361701986850116e-5

In [9]:
S = FEM.solveStress(problem, q)

LowLevelFEM.TensorField([[1.000058220699212; -2.5561726603022854e-6; … ; 0.0; 0.0;;], [1.0000632696975962; 5.327858762332876e-6; … ; 0.0; 0.0;;], [1.0000139405301542; 4.429681243592905e-6; … ; 0.0; 0.0;;], [0.7859335642834363; -0.06994854531082405; … ; 0.0; 0.0;;], [0.9995487995907826; -0.0008938111873736375; … ; 0.0; 0.0;;], [1.0135681740518414; 0.004762456389248218; … ; 0.0; 0.0;;], [0.9999761683542354; -4.938031145505274e-5; … ; 0.0; 0.0;;], [1.0002700117027234; -0.00031270188455995537; … ; 0.0; 0.0;;], [1.0006774186962515; 0.0006855838883672005; … ; 0.0; 0.0;;], [0.9999201970762527; 2.0281561039222768e-5; … ; 0.0; 0.0;;]  …  [0.06552000925673007; -0.019793344267665658; … ; 0.0; 0.0;;], [-0.02553732066118819; 0.05151403171609548; … ; 0.0; 0.0;;], [0.6065618419048677; -0.4390533857323366; … ; 0.0; 0.0;;], [-0.03085738977145422; -0.00863543756689355; … ; 0.0; 0.0;;], [-0.032289003636975566; 0.02408485129681687; … ; 0.0; 0.0;;], [0.06269648377949254; -0.01791602730680689; … ; 0.0; 0.0;

In [10]:
S1 = FEM.elementsToNodes(problem, S)

1101240×1 Matrix{Float64}:
 -7.412675537277202e-8
  1.1984391210723326e-9
  0.0
  1.1984391210723326e-9
 -1.030695238202516
  0.0
  0.0
  0.0
  0.0
  2.8688293167017287
  ⋮
  1.4708178506041354
  0.6000741028033157
  0.0
  0.6000741028033157
  0.3491976918849163
  0.0
  0.0
  0.0
  0.0

In [11]:
a = 5
N = 1
σR(x, y, z) = N * (1 - a^2 / (x^2 + y^2)) / 2 + N * cos(2 * atan(y / x)) * (3 * a^2 / (x^2 + y^2)^2 - 4 * a^2 / (x^2 + y^2) + 1) / 2
σφ(x, y, z) = N * (1 + a^2 / (x^2 + y^2)) / 2 - N * cos(2 * atan(y / x)) * (3 * a^4 / (x^2 + y^2)^2 + 1) / 2
τRφ(x, y, z) = N * sin(2 * atan(y / x)) * (3 * a^2 / (x^2 + y^2)^2 - 2 * a^2 / (x^2 + y^2) - 1) / 2

τRφ (generic function with 1 method)

In [12]:
sR = FEM.field("plate", f=σR)
sφ = FEM.field("plate", f=σφ)
sRφ = FEM.field("plate", f=τRφ)

sR1 = FEM.scalarField(problem, [sR])
sφ1 = FEM.scalarField(problem, [sφ])
sRφ1 = FEM.scalarField(problem, [sRφ])

122360-element Vector{Float64}:
 -0.0
 -0.09902950679461633
  0.09902950679461633
  0.09902950679461633
 -0.09902950679461633
 -0.05724938173957289
 -0.11440824043926864
 -0.17138619619474277
 -0.2280931551463894
 -0.28443945193625164
  ⋮
  1.1514302094453335
  1.1662489677165289
  1.1871241441546159
  1.1722862022718383
  1.1590247740015087
  1.1765901814064577
  1.1798638615517256
  1.1617700163029976
  1.1693519817799902

In [13]:
FEM.showStressResults(problem, S, :s, visible=true)

FEM.showDoFResults(problem, sR1, :scalar)
FEM.showDoFResults(problem, sφ1, :scalar)
FEM.showDoFResults(problem, sRφ1, :scalar)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [14]:
nx(x, y) = x
ny(x, y) = y
cs = FEM.CoordinateSystem([nx, ny])
Q = FEM.rotateNodes(problem, "plate", cs)

LowLevelFEM.Transformation(sparse([1, 2, 3, 4, 3, 4, 5, 6, 5, 6  …  244715, 244716, 244717, 244718, 244717, 244718, 244719, 244720, 244719, 244720], [1, 2, 3, 3, 4, 4, 5, 5, 6, 6  …  244716, 244716, 244717, 244717, 244718, 244718, 244719, 244719, 244720, 244720], [1.0, 1.0, -0.9950371902099892, -0.09950371902099892, 0.09950371902099892, -0.9950371902099892, 0.9950371902099892, -0.09950371902099892, 0.09950371902099892, 0.9950371902099892  …  -0.8550115733341447, -0.5186089176486175, -0.5111781465702527, 0.8594747829162885, -0.8594747829162885, -0.5111781465702527, -0.5173178695987247, 0.8557933289023915, -0.8557933289023915, -0.5173178695987247], 244720, 244720), 122360, 2)

In [15]:
S2 = Q' * S1 * Q;

In [16]:
e = FEM.fieldError(problem, S)

1101240×1 Matrix{Float64}:
 6.290976073408672e-11
 1.6702594688109472e-9
 0.0
 1.6702594688109472e-9
 1.294018225905802e-10
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [17]:
FEM.showDoFResults(problem, S2, :sx, name="σR")
FEM.showDoFResults(problem, S2, :sy, name="σφ")
FEM.showDoFResults(problem, S2, :sxy, name="τRφ")
FEM.showDoFResults(problem, e, :scalar, name="Error")

7

In [18]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [19]:
gmsh.finalize()